In [3]:
from PIL import Image

import numpy as np

from skimage.restoration import inpaint

def load_image(image_path):

    img = Image.open(image_path)

    return img

def mark_watermark_area(img, x1, y1, x2, y2):

    img_array = np.array(img)

    mask = np.zeros(img_array.shape[:2], dtype=np.uint8)

    mask[y1:y2, x1:x2] = 1

    return img_array, mask

def remove_watermark(img_array, mask):

    result = inpaint.inpaint_biharmonic(img_array, mask, multichannel=True)

    return result

def save_image(result, output_path):

    img_result = Image.fromarray(result.astype('uint8'))

    img_result.save(output_path)



In [4]:
# 主程序

image_path = '/Users/zouxd/Desktop/Screenshot 2025-11-14 at 13.26.11.png'

output_path = 'path/to/output/image.jpg'

x1, y1, x2, y2 = 100, 100, 200, 200

img = load_image(image_path)

img_array, mask = mark_watermark_area(img, x1, y1, x2, y2)

result = remove_watermark(img_array, mask)

save_image(result, output_path)

TypeError: inpaint_biharmonic() got an unexpected keyword argument 'multichannel'

In [ ]:
##代码test：生成斜纹半透明水印
from PIL import Image, ImageDraw, ImageFont
import math

# ========= 参数区 =========
input_image_path = "input.jpg"        # 原图路径
output_image_path = "watermarked.jpg" # 输出路径
watermark_text = "SAMPLE WATERMARK"   # 水印文字
watermark_opacity = 0.15              # 水印不透明度 alpha，0~1，建议 0.1~0.3
font_size = 40                        # 字体大小
angle_deg = 30                        # 斜纹角度（度数）
tile_spacing = 200                    # 水印之间的间距（像素）
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"  # 换成你本机的字体路径

# ========= 1. 打开原图，准备背景 B(x,y) =========
base = Image.open(input_image_path).convert("RGBA")  # RGBA 方便做 alpha 混合
W, H = base.size

# ========= 2. 创建一张“纯水印图层” =========
# 这张图相当于 W(x,y)，但先只是单色文字（白色），不含透明度
watermark_layer = Image.new("RGBA", (W, H), (0, 0, 0, 0))
draw = ImageDraw.Draw(watermark_layer)

try:
    font = ImageFont.truetype(font_path, font_size)
except IOError:
    # 如果字体路径不对，退回默认字体
    font = ImageFont.load_default()

# ========= 3. 在一个更大的画布上排布重复水印，再旋转 =========
# 为了旋转后不截掉边缘，这里做一个对角线长度的正方形画布
diag = int(math.sqrt(W*W + H*H))  # 对角线长度
tile_layer = Image.new("RGBA", (diag, diag), (0, 0, 0, 0))
tile_draw = ImageDraw.Draw(tile_layer)

# 在 tile_layer 上做网格排布的水印文字（还不旋转）
for y in range(0, diag, tile_spacing):
    for x in range(0, diag, tile_spacing):
        tile_draw.text((x, y), watermark_text, font=font, fill=(255, 255, 255, 255))

# 把这一大块水印图旋转一个角度，用 expand=True 保证完整
rotated = tile_layer.rotate(angle_deg, expand=True)

# ========= 4. 把旋转后的水印图裁剪回原图大小 =========
# 我们从旋转图的中心往外裁一个 W x H 的区域
rw, rh = rotated.size
left = (rw - W) // 2
top = (rh - H) // 2
cropped_watermark = rotated.crop((left, top, left + W, top + H))

# ========= 5. 控制整体透明度（构造 alpha(x,y)） =========
# 这里我们简单地对所有非透明像素统一乘以 watermark_opacity
# 这一步相当于给 W(x,y) 乘上一个全局 alpha
alpha_scaled = cropped_watermark.split()[3].point(
    lambda a: int(a * watermark_opacity)
)
watermark_with_alpha = cropped_watermark.copy()
watermark_with_alpha.putalpha(alpha_scaled)

# ========= 6. 把水印层叠加到原图上，得到 I_obs(x,y) =========
# PIL 的 alpha_composite 做的就是：
# I_obs = alpha * W + (1 - alpha) * B
composited = Image.alpha_composite(base, watermark_with_alpha)

# 如果你想保存成 JPG，可以去掉 alpha 通道
composited_rgb = composited.convert("RGB")
composited_rgb.save(output_image_path, quality=95)

print("Done! Saved to:", output_image_path)
